In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/california_housing_test.csv')
# For Parquet: df = pd.read_parquet('path/to/parquet')
#print(df.dtypes)
display(df.head(10))

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
5,-119.56,36.51,37.0,1018.0,213.0,663.0,204.0,1.6635,67000.0
6,-121.43,38.63,43.0,1009.0,225.0,604.0,218.0,1.6641,67000.0
7,-120.65,35.48,19.0,2310.0,471.0,1341.0,441.0,3.2250,166900.0
8,-122.84,38.40,15.0,3080.0,617.0,1446.0,599.0,3.6696,194400.0
9,-118.02,34.08,31.0,2402.0,632.0,2830.0,603.0,2.3333,164200.0


In [9]:
# Convert 'total_rooms' to integer type
df_clean = df.copy() # Create a copy to avoid modifying the original DataFrame
df_clean['total_rooms'] = df_clean['total_rooms'].astype(int)

df_clean = df_clean[df_clean['median_house_value'] > 100000] \
          .dropna() \
          .sort_values(by='housing_median_age', ascending=False)

# Drop duplicate rows based on 'housing_median_age', keeping the first occurrence
df_clean = df_clean.drop_duplicates(subset=['housing_median_age'], keep='first')

# Add a new column 'duplex_room'
df_clean['duplex_room'] = df_clean['total_rooms'] * 2
display(df_clean.head(10))

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,duplex_room
993,-122.29,37.85,52.0,477,119.0,218.0,106.0,2.5682,120000.0,954
2900,-121.45,38.56,51.0,1250,235.0,452.0,232.0,2.6250,121200.0,2500
223,-118.27,34.10,50.0,2113,398.0,793.0,418.0,4.7132,304600.0,4226
2497,-122.32,37.56,49.0,2016,299.0,691.0,288.0,5.5490,500001.0,4032
2658,-121.89,37.35,48.0,1562,439.0,1469.0,424.0,2.5673,177500.0,3124
1911,-118.26,34.11,47.0,2183,510.0,1445.0,503.0,3.6667,210900.0,4366
1895,-121.51,38.55,46.0,1485,278.0,531.0,291.0,2.7885,137200.0,2970
2875,-118.21,34.05,45.0,2146,607.0,2868.0,625.0,2.1210,144000.0,4292
2911,-121.80,38.01,44.0,3184,581.0,1399.0,548.0,2.7234,110200.0,6368
974,-117.23,32.72,43.0,952,209.0,392.0,210.0,2.1635,244200.0,1904


In [13]:
import pandas as pd
df_train = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Perform an inner join on 'latitude' and 'longitude' using pandas merge
df_joined = pd.merge(df, df_train, on=['latitude', 'longitude'], how='inner')

# Group by 'housing_median_age' and calculate the sum of 'median_house_value'
agg_df = df_joined.groupby('housing_median_age_y')['median_house_value_y'].agg('sum').reset_index()
agg_df = agg_df.rename(columns={'housing_median_age_y': 'housing_median_age', 'median_house_value_y': 'sum_median_house_value'})

display(agg_df.head(10))

,housing_median_age,sum_median_house_value
0,2.0,860400.0
1,3.0,1139300.0
2,4.0,1957200.0
3,5.0,3138600.0
4,6.0,1246600.0
5,7.0,568000.0
6,8.0,1687000.0
7,9.0,3039300.0
8,10.0,4457600.0
9,11.0,1887500.0


In [14]:
# Window Functions - Calculate the rank of median_house_value
df_joined['rank'] = df_joined.groupby(df_joined.housing_median_age_y)['median_house_value_y'].rank(method='first', ascending=False)

# Select all columns from df_train and the 'rank' column
# The columns from df_train in df_joined have a '_y' suffix
df_final_cols = [col for col in df_joined.columns if col.endswith('_y')] + ['latitude', 'longitude', 'rank']
df_final = df_joined[df_final_cols]

# Rename the columns from df_train by removing the '_y' suffix
df_final = df_final.rename(columns=lambda x: x.replace('_y', '') if x.endswith('_y') else x)

display(df_final.head(10))

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,latitude,longitude,rank
0,35.0,1365.0,256.0,662.0,262.0,5.6533,291400.0,37.37,-122.05,24.0
1,27.0,2687.0,768.0,1362.0,725.0,3.4028,324200.0,37.37,-122.05,11.0
2,40.0,1065.0,214.0,605.0,183.0,4.1964,185900.0,34.26,-118.30,34.0
3,28.0,1643.0,489.0,1142.0,458.0,3.1607,200600.0,34.26,-118.30,39.0
4,36.0,1784.0,311.0,901.0,293.0,6.2247,339000.0,33.82,-118.36,13.0
5,36.0,1083.0,187.0,522.0,187.0,5.7765,339500.0,33.82,-118.36,11.0
6,29.0,2741.0,667.0,2449.0,677.0,3.6944,175200.0,34.08,-118.02,38.0
7,45.0,173.0,42.0,230.0,57.0,3.0724,110700.0,33.98,-118.24,72.0
8,37.0,1196.0,364.0,1622.0,327.0,2.1250,108900.0,33.98,-118.24,110.0
9,30.0,861.0,250.0,1062.0,231.0,1.7500,115400.0,33.98,-118.24,73.0
